In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
# Specify the subdirectories you want to loop through
subdirs = ['1000l_r1', '1000l_r2', '1000l_r3', '1000l_r4','1000l_r5','1000l_r6']  # List of subdirectories you want to loop through
file_posfix='1000_20ns'

In [3]:
def extract_data(file_name):
    # Specify the subdirectories you want to loop through
    dict_name = {}  # Dictionary to store dict_name

    # Loop through each subdirectory
    for subdir in subdirs:
        length=(len(subdir))
        n_rep=3
        newname=subdir[length-n_rep:]
        # Construct the path to the CSV file (assuming the CSV has the same name as the subdir)
        file_path = os.path.join(os.getcwd(), subdir, f'{file_name}')

        # Read the CSV into a pandas dataframe
        if os.path.exists(file_path):
            if '.csv' in file_name:
                df = pd.read_csv(file_path)
                dict_name[f'df_stats{newname}'] = df
            elif '.json' in file_name:    
                df = pd.read_json(file_path)
            # Store the dataframe with a unique name (using the subdirectory name)
                if 'excess' in file_name:
                    dict_name[f'value{newname}'] = df['.result']['value']['value']
                    dict_name[f'error{newname}'] = df['.result']['value']['error']
                elif 'build' in file_name:
                    dict_name[f'molecule_number{newname}'] = df['.output_number_of_molecules'][0]
                    dict_name[f'molefraction_0{newname}'] = df['.output_substance']['amounts']['O{solv}'][0]['value']
                    dict_name[f'molefraction_1{newname}'] = df['.output_substance']['amounts']['OCCN(CCO)CCO{solv}'][0]['value']
                else:
                    dict_name[f'value{newname}'] = df['.value']['value']['value']
                    dict_name[f'error{newname}'] = df['.value']['value']['error']
                    dict_name[f'n_total_points{newname}'] = df['.time_series_statistics']["n_total_points"]
                    dict_name[f'n_uncorrelated_points{newname}'] = df['.time_series_statistics']["n_uncorrelated_points"]
                    dict_name[f'statistical_inefficiency{newname}'] = df['.time_series_statistics']["statistical_inefficiency"]
                    dict_name[f'equilibration_index{newname}'] = df['.time_series_statistics']["equilibration_index"]

            print(f"Data from {subdir} loaded successfully into dataframe: {subdir}")
        else:
            print(f"Data not found in {subdir}")
    return dict_name

## SYSTEM

In [4]:
system=extract_data('6421_build_coordinates_mixture_output.json')

Data from 1000l_r1 loaded successfully into dataframe: 1000l_r1
Data from 1000l_r2 loaded successfully into dataframe: 1000l_r2
Data from 1000l_r3 loaded successfully into dataframe: 1000l_r3
Data from 1000l_r4 loaded successfully into dataframe: 1000l_r4
Data from 1000l_r5 loaded successfully into dataframe: 1000l_r5
Data from 1000l_r6 loaded successfully into dataframe: 1000l_r6


In [5]:
def are_values_of_keys_equal(dictionary, keys):
    # Extract the values of the specified keys from the dictionary
    values = [dictionary.get(key) for key in keys]
    
    # Check if all values are the same
    return all(value == values[0] for value in values)
        

keys_to_check_molnumber = []
keys_to_check_mf0 = []
keys_to_check_mf1 = []

for key,val in system.items():
    for rep in range(1,7):
        if 'molecule_number' in key:
            keys_to_check_molnumber.append(f'{key[:-1]}{rep}')

        if 'molefraction_0' in key:
            keys_to_check_mf0.append(f'{key[:-1]}{rep}')

        if 'molefraction_1' in key:
            keys_to_check_mf1.append(f'{key[:-1]}{rep}')
            
print(are_values_of_keys_equal(system, keys_to_check_molnumber))
print(are_values_of_keys_equal(system, keys_to_check_mf0))
print(are_values_of_keys_equal(system, keys_to_check_mf1))

True
True
True


In [6]:
print(system['molecule_number_r1'])
print(system['molefraction_0_r1'])
print(system['molefraction_1_r1'])

1000
0.51
0.49


In [7]:
thermoml_MF_c1=0.4902
thermoml_MF_c0=0.5098
evaluator_MF_c1=system['molefraction_1_r1']
evaluator_MF_c0=system['molefraction_0_r1']

## ENERGY STATS

In [8]:
stats=extract_data('openmm_statistics.csv')

Data from 1000l_r1 loaded successfully into dataframe: 1000l_r1
Data from 1000l_r2 loaded successfully into dataframe: 1000l_r2
Data from 1000l_r3 loaded successfully into dataframe: 1000l_r3
Data from 1000l_r4 loaded successfully into dataframe: 1000l_r4
Data from 1000l_r5 loaded successfully into dataframe: 1000l_r5
Data from 1000l_r6 loaded successfully into dataframe: 1000l_r6


In [9]:
means_stats={}
means_stats['rep1'] = stats['df_stats_r1'].mean()
means_stats['rep2'] = stats['df_stats_r2'].mean()
means_stats['rep3'] = stats['df_stats_r3'].mean()
means_stats['rep4'] = stats['df_stats_r4'].mean()
means_stats['rep5'] = stats['df_stats_r5'].mean()
means_stats['rep6'] = stats['df_stats_r6'].mean()

In [10]:
PE=[]
KE=[]
TE=[]
bVol=[]
dens=[]

for key, value_list in means_stats.items():
    print(f'Key: {key}')
    # print(f'Values: {value_list}')
    PE.append(value_list.iloc[1])
    KE.append(value_list.iloc[2])
    TE.append(value_list.iloc[3])
    bVol.append(value_list.iloc[5])
    dens.append(value_list.iloc[6])


Key: rep1
Key: rep2
Key: rep3
Key: rep4
Key: rep5
Key: rep6


In [11]:
## Calculate means and MSE
PE_mean=np.array(PE).mean()
PE_std=np.array(PE).std(ddof=1)
PE_err=PE_std/np.sqrt(len(PE))
print(f'Hmix replicates, Potential Energy (kJ/mol): {PE_mean:.3f} +/- {PE_err:.3f}')

KE_mean=np.array(KE).mean()
KE_std=np.array(KE).std(ddof=1)
KE_err=KE_std/np.sqrt(len(KE))
print(f'Hmix replicates, Kinetic Energy (kJ/mol): {KE_mean:.3f} +/- {KE_err:.3f}')

TE_mean=np.array(TE).mean()
TE_std=np.array(TE).std(ddof=1)
TE_err=TE_std/np.sqrt(len(TE))
print(f'Hmix replicates, Total Energy (kJ/mol): {TE_mean:.3f} +/- {TE_err:.3f}')

bVol_mean=np.array(bVol).mean()
bVol_std=np.array(bVol).std(ddof=1)
bVol_err=bVol_std/np.sqrt(len(bVol))
print(f'Hmix replicates, Box Volume (nm^3): {bVol_mean:.3f} +/- {bVol_err:.3f}')

dens_mean=np.array(dens).mean()
dens_std=np.array(dens).std(ddof=1)
dens_err=dens_std/np.sqrt(len(dens))
print(f'Hmix replicates, Density (g/mL): {dens_mean:.3f} +/- {dens_err:.3f}')

Hmix replicates, Potential Energy (kJ/mol): 59951.433 +/- 59.593
Hmix replicates, Kinetic Energy (kJ/mol): 39317.899 +/- 2.373
Hmix replicates, Total Energy (kJ/mol): 99269.332 +/- 59.942
Hmix replicates, Box Volume (nm^3): 120.481 +/- 0.063
Hmix replicates, Density (g/mL): 1.134 +/- 0.001


In [12]:
molnumb=system['molecule_number_r1']

In [13]:
with open(f'output_stats_{file_posfix}.txt', 'w') as file:
    file.write(f'Molecule number: {molnumb} \n')
    file.write(f'Hmix {file_posfix} replicates, Average Potential Energy (kJ/mol): {PE_mean:.3f} +/- {PE_err:.3f} \n')
    file.write(f'Hmix {file_posfix} replicates, Average Kinetic Energy (kJ/mol): {KE_mean:.3f} +/- {KE_err:.3f} \n')
    file.write(f'Hmix {file_posfix} replicates, Average Total Energy (kJ/mol): {TE_mean:.3f} +/- {TE_err:.3f} \n')
    file.write(f'Hmix {file_posfix} replicates, Average Box Volume (nm^3): {bVol_mean:.3f} +/- {bVol_err:.3f} \n')
    file.write(f'Hmix {file_posfix} replicates, Average Density (g/mL): {dens_mean:.3f} +/- {dens_err:.3f} \n')

## FINAL RESULTS

In [14]:
final_results=extract_data('6421_calculate_excess_observable_output.json')

Data from 1000l_r1 loaded successfully into dataframe: 1000l_r1
Data from 1000l_r2 loaded successfully into dataframe: 1000l_r2
Data from 1000l_r3 loaded successfully into dataframe: 1000l_r3
Data from 1000l_r4 loaded successfully into dataframe: 1000l_r4
Data from 1000l_r5 loaded successfully into dataframe: 1000l_r5
Data from 1000l_r6 loaded successfully into dataframe: 1000l_r6


In [15]:
obs_values=[]
obs_errors=[]
for key, mr in final_results.items():
    if 'value' in key:
        print(f"{key}:{mr['value']}")
        obs_values.append(mr['value'])
    elif 'error' in key:
        print(f"{key}:{mr['value']}")
        obs_errors.append(mr['value'])

value_r1:-1.00438788652093
error_r1:0.027793276305126003
value_r2:-1.359631594605289
error_r2:0.029301131494792004
value_r3:-1.394975354843708
error_r3:0.030597590011105003
value_r4:-1.279081061707629
error_r4:0.037290156173098006
value_r5:-0.975410929143194
error_r5:0.036557577563663006
value_r6:-1.511265698933385
error_r6:0.038797612410837005


In [16]:
with open(f'output_observables_{file_posfix}.txt', 'w') as file:
    file.write(f'Values list: {obs_values} \n')
    file.write(f'Errors list: {obs_errors} \n')

## MIXTURE

In [17]:
mixout=extract_data('6421_extract_observable_mixture_output.json')

Data from 1000l_r1 loaded successfully into dataframe: 1000l_r1
Data from 1000l_r2 loaded successfully into dataframe: 1000l_r2
Data from 1000l_r3 loaded successfully into dataframe: 1000l_r3
Data from 1000l_r4 loaded successfully into dataframe: 1000l_r4
Data from 1000l_r5 loaded successfully into dataframe: 1000l_r5
Data from 1000l_r6 loaded successfully into dataframe: 1000l_r6


In [18]:
mixout_values=[]
mixout_errors=[]
mixout_totpts=[]
mixout_uncorrpts=[]
mixout_statineff=[]
mixout_equilindex=[]

for key, mr in mixout.items():
    if 'value' in key:
        print(f"{key}:{mr['value']}")
        mixout_values.append(mr['value'])
    elif 'error' in key:
        print(f"{key}:{mr['value']}")
        mixout_errors.append(mr['value'])
    elif 'n_total_points' in key:
        print(f"{key}:{mr}")
        mixout_totpts.append(mr)
    elif 'n_uncorrelated_points' in key:
        print(f"{key}:{mr}")
        mixout_uncorrpts.append(mr)
    elif 'statistical_inefficiency' in key:
        print(f"{key}:{mr}")
        mixout_statineff.append(mr)
    elif 'equilibration_index' in key:
        print(f"{key}:{mr}")
        mixout_equilindex.append(mr)

value_r1:99.41846722926692
error_r1:0.019630000121839002
n_total_points_r1:5000
n_uncorrelated_points_r1:960
statistical_inefficiency_r1:1.919515007030599
equilibration_index_r1:3081
value_r2:99.03433328033205
error_r2:0.026266070430326003
n_total_points_r2:5000
n_uncorrelated_points_r2:450
statistical_inefficiency_r2:2.028505800425906
equilibration_index_r2:3651
value_r3:99.10532005743546
error_r3:0.029030830654064
n_total_points_r3:5000
n_uncorrelated_points_r3:275
statistical_inefficiency_r3:1.092257748264158
equilibration_index_r3:4450
value_r4:99.27380702714406
error_r4:0.028834582186391
n_total_points_r4:5000
n_uncorrelated_points_r4:281
statistical_inefficiency_r4:1.399651882184411
equilibration_index_r4:4438
value_r5:99.56847109367975
error_r5:0.031823695662625
n_total_points_r5:5000
n_uncorrelated_points_r5:203
statistical_inefficiency_r5:1.008136724567934
equilibration_index_r5:4595
value_r6:98.85450603530933
error_r6:0.036566791633431
n_total_points_r6:5000
n_uncorrelated_po

In [19]:
with open(f'mixture_output_{file_posfix}.txt', 'w') as file:
    file.write(f'Mixture Values list: {mixout_values} \n')
    file.write(f'Mixture Errors list: {mixout_errors} \n')
    file.write(f'Mixture Total points list: {mixout_totpts} \n')
    file.write(f'Mixture Uncorrelated points list: {mixout_uncorrpts} \n')
    file.write(f'Mixture Statistical inefficiency list: {mixout_statineff} \n')
    file.write(f'Mixture Equilibration index list: {mixout_equilindex} \n')

## COMPONENT 0

In [20]:
comp0=extract_data('6422_extract_observable_component_0_output.json')

Data from 1000l_r1 loaded successfully into dataframe: 1000l_r1
Data from 1000l_r2 loaded successfully into dataframe: 1000l_r2
Data from 1000l_r3 loaded successfully into dataframe: 1000l_r3
Data from 1000l_r4 loaded successfully into dataframe: 1000l_r4
Data from 1000l_r5 loaded successfully into dataframe: 1000l_r5
Data from 1000l_r6 loaded successfully into dataframe: 1000l_r6


In [21]:
comp0_values=[]
comp0_errors=[]
comp0_totpts=[]
comp0_uncorrpts=[]
comp0_statineff=[]
comp0_equilindex=[]

for key, mr in comp0.items():
    if 'value' in key:
        print(f"{key}:{mr['value']}")
        comp0_values.append(mr['value'])
    elif 'error' in key:
        print(f"{key}:{mr['value']}")
        comp0_errors.append(mr['value'])
    elif 'n_total_points' in key:
        print(f"{key}:{mr}")
        comp0_totpts.append(mr)
    elif 'n_uncorrelated_points' in key:
        print(f"{key}:{mr}")
        comp0_uncorrpts.append(mr)
    elif 'statistical_inefficiency' in key:
        print(f"{key}:{mr}")
        comp0_statineff.append(mr)
    elif 'equilibration_index' in key:
        print(f"{key}:{mr}")
        comp0_equilindex.append(mr)

value_r1:-44.87094396463476
error_r1:0.005241095523625
n_total_points_r1:5000
n_uncorrelated_points_r1:2500
statistical_inefficiency_r1:1.563413544543261
equilibration_index_r1:0
value_r2:-44.87249062157608
error_r2:0.005503189687938
n_total_points_r2:5000
n_uncorrelated_points_r2:2500
statistical_inefficiency_r2:1.421293642724092
equilibration_index_r2:0
value_r3:-44.87572711227428
error_r3:0.005703782443673
n_total_points_r3:5000
n_uncorrelated_points_r3:2496
statistical_inefficiency_r3:1.488460556606125
equilibration_index_r3:9
value_r4:-44.88168396954277
error_r4:0.00570196725159
n_total_points_r4:5000
n_uncorrelated_points_r4:2500
statistical_inefficiency_r4:1.426474810561663
equilibration_index_r4:0
value_r5:-44.88708656250604
error_r5:0.0057649258555510004
n_total_points_r5:5000
n_uncorrelated_points_r5:2499
statistical_inefficiency_r5:1.457967519409496
equilibration_index_r5:2
value_r6:-44.87892946171502
error_r6:0.005199250522814
n_total_points_r6:5000
n_uncorrelated_points_r6

In [22]:
with open(f'Component0_output_{file_posfix}.txt', 'w') as file:
    file.write(f'Component0 Values list: {comp0_values} \n')
    file.write(f'Component0 Errors list: {comp0_errors} \n')
    file.write(f'Component0 Total points list: {comp0_totpts} \n')
    file.write(f'Component0 Uncorrelated points list: {comp0_uncorrpts} \n')
    file.write(f'Component0 Statistical inefficiency list: {comp0_statineff} \n')
    file.write(f'Component0 Equilibration index list: {comp0_equilindex} \n')

## COMPONENT 1

In [23]:
comp1=extract_data('6422_extract_observable_component_1_output.json')

Data from 1000l_r1 loaded successfully into dataframe: 1000l_r1
Data from 1000l_r2 loaded successfully into dataframe: 1000l_r2
Data from 1000l_r3 loaded successfully into dataframe: 1000l_r3
Data from 1000l_r4 loaded successfully into dataframe: 1000l_r4
Data from 1000l_r5 loaded successfully into dataframe: 1000l_r5
Data from 1000l_r6 loaded successfully into dataframe: 1000l_r6


In [24]:
comp1_values=[]
comp1_errors=[]
comp1_totpts=[]
comp1_uncorrpts=[]
comp1_statineff=[]
comp1_equilindex=[]

for key, mr in comp1.items():
    if 'value' in key:
        print(f"{key}:{mr['value']}")
        comp1_values.append(mr['value'])
    elif 'error' in key:
        print(f"{key}:{mr['value']}")
        comp1_errors.append(mr['value'])
    elif 'n_total_points' in key:
        print(f"{key}:{mr}")
        comp1_totpts.append(mr)
    elif 'n_uncorrelated_points' in key:
        print(f"{key}:{mr}")
        comp1_uncorrpts.append(mr)
    elif 'statistical_inefficiency' in key:
        print(f"{key}:{mr}")
        comp1_statineff.append(mr)
    elif 'equilibration_index' in key:
        print(f"{key}:{mr}")
        comp1_equilindex.append(mr)

value_r1:251.52603498359576
error_r1:0.039766089868522
n_total_points_r1:5000
n_uncorrelated_points_r1:423
statistical_inefficiency_r1:1.279845018945329
equilibration_index_r1:4154
value_r2:251.4687078617234
error_r2:0.025866709452493003
n_total_points_r2:5000
n_uncorrelated_points_r2:941
statistical_inefficiency_r2:1.656688179782199
equilibration_index_r2:3119
value_r3:251.68898632010732
error_r3:0.018804196582712002
n_total_points_r3:5000
n_uncorrelated_points_r3:1173
statistical_inefficiency_r3:1.918600363318369
equilibration_index_r3:2654
value_r4:251.80246955635369
error_r4:0.047871109396957004
n_total_points_r4:5000
n_uncorrelated_points_r4:307
statistical_inefficiency_r4:2.437083402835697
equilibration_index_r4:4081
value_r5:251.78971593714508
error_r5:0.036210221205892006
n_total_points_r5:5000
n_uncorrelated_points_r5:361
statistical_inefficiency_r5:3.255169164099932
equilibration_index_r5:3557
value_r6:251.4178906034782
error_r6:0.025892441338233003
n_total_points_r6:5000
n_u

In [25]:
with open(f'Component1_output_{file_posfix}.txt', 'w') as file:
    file.write(f'Component1 Values list: {comp1_values} \n')
    file.write(f'Component1 Errors list: {comp1_errors} \n')
    file.write(f'Component1 Total points list: {comp1_totpts} \n')
    file.write(f'Component1 Uncorrelated points list: {comp1_uncorrpts} \n')
    file.write(f'Component1 Statistical inefficiency list: {comp1_statineff} \n')
    file.write(f'Component1 Equilibration index list: {comp1_equilindex} \n')

## ANALYSIS

In [26]:
def estimated_values(mole_fraction_source):
    if mole_fraction_source == 'thermoML':
        mf_c0=thermoml_MF_c0
        mf_c1=thermoml_MF_c1
    elif mole_fraction_source == 'evaluator':
        mf_c0=evaluator_MF_c0
        mf_c1=evaluator_MF_c1

    for i in range(6):
        estimated=mixout_values[i]-(mf_c1*comp1_values[i]+mf_c0*comp0_values[i])
        est_error=np.sqrt((mf_c1*comp1_errors[i])**2 + (mf_c0*comp0_errors[i])**2 + mixout_errors[i]**2)
        print(f'rep #{i+1}: {estimated} +/- {est_error}')

In [27]:
estimated_values('thermoML')

rep #1: -1.0043878865209308 +/- 0.027793276305125365
rep #2: -1.3596315946052897 +/- 0.029301131494791976
rep #3: -1.3949753548437087 +/- 0.03059759001110494
rep #4: -1.279081061707629 +/- 0.037290156173097846
rep #5: -0.9754109291431945 +/- 0.03655757756366293
rep #6: -1.5112656989333857 +/- 0.03879761241083635


In [28]:
estimated_values('evaluator')

rep #1: -0.9451084907312719 +/- 0.027787799554846324
rep #2: -1.3003633549086118 +/- 0.029298998550718536
rep #3: -1.335662412157248 +/- 0.030596565667264243
rep #4: -1.219744231002423 +/- 0.03728422085726234
rep #5: -0.9160755686432509 +/- 0.036554154511555574
rep #6: -1.452006334920327 +/- 0.03879598965645626
